In [1]:
from func.shopee_crawler import search_get_items,search_get_items_with_selected_page, get_items_from_industry, get_in4_item, get_full_in4_item, get_item_price
import os
import pandas as pd

c:\Users\hoan.pc\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\hoan.pc\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\hoan.pc\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
brand_source = os.path.abspath('data\list_requested_brands.xlsx')
# read sheet
# pd.ExcelFile(brand_source).sheet_names
brand_list= [i.replace(' ','%20').lower() for i in list(pd.read_excel(brand_source,'list_brand')['brand'])]

In [3]:
# Lấy danh sách item từ mục tìm kiếm
path = r'D:\KiotvietPlus\KiotPro\PROJECT\Shopee-Crawler\data\brand_id.xlsx'
data = pd.DataFrame()
for i in brand_list[9:10]:
    page_number=0
    while True:
        try:
            read_data = pd.read_excel(path)
        except:
            read_data = pd.DataFrame()
        url = f'https://shopee.vn/search?category=11036279&filters=5&keyword={i}&sortBy=sales'
        raw_data = search_get_items_with_selected_page(url=url, page_number=page_number)
        raw_data['f_brand'] = i
        data = pd.concat([data, raw_data], axis=0)
        pd.concat([read_data,raw_data],axis=0).reset_index().drop('index',axis=1).to_excel(path, index=False)
        if data[data['f_brand']==i]['name'].nunique()>100 or page_number>3:
            break
        else:
            page_number +=1
data = data.reset_index().drop('index', axis=1)

In [14]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 102 columns):
 #    Column                                                           Dtype  
---   ------                                                           -----  
 0    itemid                                                           float64
 1    shopid                                                           float64
 2    name                                                             object 
 3    label_ids                                                        object 
 4    image                                                            object 
 5    images                                                           object 
 6    currency                                                         object 
 7    stock                                                            float64
 8    status                                                           float64
 9    ctime                

In [12]:
headers = {
'authority': 'shopee.vn',
'accept': '*/*',
'accept-language': 'en-US,en;q=0.9,vi;q=0.8',
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
'x-api-source': 'pc',
'x-kl-ajax-request': 'Ajax_Request',
'x-requested-with': 'XMLHttpRequest',
'x-shopee-language': 'vi'
}
import requests
# def get_item_price(df):
#     item_url = f"https://shopee.vn/api/v4/product/get_purchase_quantities_for_selected_model?itemId={df['itemid']}&modelId={df['modelid']}&shopId={df['shopid']}"

#     payload={
#     }
#     headers = {
#     'authority': 'shopee.vn',
#     'accept': '*/*',
#     'accept-language': 'en-US,en;q=0.9,vi;q=0.8',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
#     'x-api-source': 'pc',
#     'x-kl-ajax-request': 'Ajax_Request',
#     'x-requested-with': 'XMLHttpRequest',
#     'x-shopee-language': 'vi'
#     }
#     response = requests.request("GET", item_url, headers=headers, data=payload)

#     item_raw=response.json()
#     price_r_data = pd.json_normalize(item_raw['available_price_and_stocks'])
#     price_r_data = price_r_data[price_r_data.display_price >0]
#     min_price = price_r_data[price_r_data.display_price == price_r_data.display_price.min()]
#     price_data = pd.concat([pd.json_normalize(item_raw)[['price_before_discount']],min_price], axis = 1)
#     price_data = price_data[['price_before_discount','display_stock','display_price']]
#     return price_data

def flat_dict(data):
    df = pd.DataFrame(data)
    unnest = df.explode('categories')
    list_cate = [i['display_name'] for i in unnest['categories']]
    cate_df = pd.concat([data,pd.DataFrame(list_cate).T], axis=1)
    return cate_df


# Thông tin cơ bản sản phẩm
def get_in4_item1(shopid: str, itemid: str):
    item_url = f"https://shopee.vn/api/v4/item/get?itemid={itemid}&shopid={shopid}"

    payload={
    }
    headers = {
    'authority': 'shopee.vn',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,vi;q=0.8',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
    'x-api-source': 'pc',
    'x-kl-ajax-request': 'Ajax_Request',
    'x-requested-with': 'XMLHttpRequest',
    'x-shopee-language': 'vi'
    }
    response = requests.request("GET", item_url, headers=headers, data=payload)

    item_raw=response.json()
    # overview product infor
    product_data=pd.json_normalize(item_raw['data'])
    inf = product_data[['shopid','itemid','name','brand','categories','historical_sold','item_rating.rating_star','images']]#'price','stock','discount','historical_sold','sold','description','categories','shop_vouchers','global_sold'
    inf = pd.concat([inf,flat_dict(inf['categories'])],axis=1).rename(columns={'name':'parent_name'}).drop('categories',axis=1)
    inf = inf.astype({'itemid':str,
                      'shopid':str})
    
    # Detail product infor
    model_data = pd.json_normalize(item_raw['data']['models'])[['itemid','name','modelid']].assign(shopid=shopid)
    # get_price = pd.concat(model_data.apply(lambda x: get_item_price(x), axis=1).tolist(), ignore_index=True)
    # price_data = pd.concat([model_data,get_price],axis=1).rename(columns={'itemid':'item','shopid':'shop'})
    # price_data = price_data.astype({'item':str,
    #                   'shop':str})
    # df = price_data.merge(inf, how='left', left_on=['item','shop'],right_on=['itemid','shopid'])
    # df = df[df['itemid'].notnull()].reset_index().drop('index',axis=1)
    return model_data

In [26]:
#Lấy thông tin chi tiết brand và giá
detail_data = pd.DataFrame()
list_item = data[['shopid','itemid']]
for i in list(range(len(list_item))):
    shopid = int(list_item.iloc[i]['shopid'])
    itemid = int(list_item.iloc[i]['itemid'])
    try:
        raw = get_in4_item(shopid,itemid)
        detail_data = pd.concat([detail_data,raw],axis=0)
    except:
        pass
detail_data = detail_data.reset_index().drop('index', axis=1)
detail_data = detail_data.drop(['shopid','itemid'],axis=1)
# Merge
final = detail_data.merge(data[['shopid','itemid','f_brand']], how='left', left_on = ['shop','item'], right_on = ['shopid','itemid'])
final['price_before_discount'] = final['price_before_discount']/1e5
final['display_price'] = final['display_price']/1e5

In [46]:
t = detail_data.iloc[[0]]

In [47]:
t

,item,name,modelid,shop,price_before_discount,display_stock,display_price,shopid,itemid,parent_name,brand,historical_sold,item_rating.rating_star,images,0,1,2,3
0,1.845344e+10,,2.126842e+11,874863209.0,3.600000e+10,10449.0,1.800000e+10,874863209.0,1.845344e+10,Nước Hoa Nam chính hãng C-Parfum Ysl y edp 20m...,Cparfum,16109.0,3.235479,"[sg-11134201-23030-vhji5k5a0sov52, sg-11134201...",Sắc Đẹp,Nước hoa,NaN,NaN


In [48]:
t['item'] = t['item'].map('{:.0f}'.format)
t['shop'] = t['shop'].map('{:.0f}'.format)

C:\Users\hoan.pc\AppData\Local\Temp\ipykernel_21744\1365958439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['item'] = t['item'].map('{:.0f}'.format)
C:\Users\hoan.pc\AppData\Local\Temp\ipykernel_21744\1365958439.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['shop'] = t['shop'].map('{:.0f}'.format)


In [49]:
t

,item,name,modelid,shop,price_before_discount,display_stock,display_price,shopid,itemid,parent_name,brand,historical_sold,item_rating.rating_star,images,0,1,2,3
0,18453443639,,2.126842e+11,874863209,3.600000e+10,10449.0,1.800000e+10,874863209.0,1.845344e+10,Nước Hoa Nam chính hãng C-Parfum Ysl y edp 20m...,Cparfum,16109.0,3.235479,"[sg-11134201-23030-vhji5k5a0sov52, sg-11134201...",Sắc Đẹp,Nước hoa,NaN,NaN


In [51]:
final

,item,name,modelid,shop,price_before_discount,display_stock,display_price,shopid_x,itemid_x,parent_name,...,historical_sold,item_rating.rating_star,images,0,1,2,3,shopid_y,itemid_y,f_brand
0,1.845344e+10,,2.126842e+11,874863209.0,360000.0,10449.0,180000.0,874863209.0,1.845344e+10,Nước Hoa Nam chính hãng C-Parfum Ysl y edp 20m...,...,16109.0,3.235479,"[sg-11134201-23030-vhji5k5a0sov52, sg-11134201...",Sắc Đẹp,Nước hoa,NaN,NaN,874863209.0,1.845344e+10,ysl
1,1.985345e+10,,1.075521e+11,874863209.0,360000.0,7389.0,180000.0,874863209.0,1.985345e+10,Nước Hoa Santal 33 chính hãng C-Parfum 20ml th...,...,10338.0,4.349232,"[sg-11134201-23030-8ikl1j5a0sov50, sg-11134201...",Sắc Đẹp,Nước hoa,NaN,NaN,874863209.0,1.985345e+10,ysl
2,2.371280e+10,5ml,9.856784e+10,923309443.0,335000.0,0.0,335000.0,923309443.0,2.371280e+10,Nước Hoa Nam YSL Y edp chính hãng Le Clapotis ...,...,15334.0,3.870979,"[vn-11134201-23030-834l8ga3luov4d, sg-11134201...",Sắc Đẹp,Nước hoa,NaN,NaN,923309443.0,2.371280e+10,ysl
3,2.371280e+10,30ml,1.466547e+11,923309443.0,590000.0,99.0,285000.0,923309443.0,2.371280e+10,Nước Hoa Nam YSL Y edp chính hãng Le Clapotis ...,...,15334.0,3.870979,"[vn-11134201-23030-834l8ga3luov4d, sg-11134201...",Sắc Đẹp,Nước hoa,NaN,NaN,923309443.0,2.371280e+10,ysl
4,2.371280e+10,10ml,1.835952e+11,923309443.0,210000.0,94.0,94000.0,923309443.0,2.371280e+10,Nước Hoa Nam YSL Y edp chính hãng Le Clapotis ...,...,15334.0,3.870979,"[vn-11134201-23030-834l8ga3luov4d, sg-11134201...",Sắc Đẹp,Nước hoa,NaN,NaN,923309443.0,2.371280e+10,ysl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,1.900062e+10,AZURE PINK,1.542488e+11,285380605.0,456000.0,0.0,456000.0,285380605.0,1.900062e+10,Son thỏi 3CE vỏ trong suốt 3CE Soft Matte Lips...,...,3038.0,4.896748,"[53efcec39992f4ba59c6aa1a1a58db69, 9029d9c4aff...",Sắc Đẹp,Trang điểm,Trang điểm môi,Son môi,285380605.0,1.900062e+10,ysl
106,1.900062e+10,BREEZY PINK,1.542488e+11,285380605.0,456000.0,14.0,456000.0,285380605.0,1.900062e+10,Son thỏi 3CE vỏ trong suốt 3CE Soft Matte Lips...,...,3038.0,4.896748,"[53efcec39992f4ba59c6aa1a1a58db69, 9029d9c4aff...",Sắc Đẹp,Trang điểm,Trang điểm môi,Son môi,285380605.0,1.900062e+10,ysl
107,1.900062e+10,PALE POSY,1.542488e+11,285380605.0,456000.0,38.0,456000.0,285380605.0,1.900062e+10,Son thỏi 3CE vỏ trong suốt 3CE Soft Matte Lips...,...,3038.0,4.896748,"[53efcec39992f4ba59c6aa1a1a58db69, 9029d9c4aff...",Sắc Đẹp,Trang điểm,Trang điểm môi,Son môi,285380605.0,1.900062e+10,ysl
108,1.900062e+10,ANOTHER WISH,1.542488e+11,285380605.0,456000.0,39.0,456000.0,285380605.0,1.900062e+10,Son thỏi 3CE vỏ trong suốt 3CE Soft Matte Lips...,...,3038.0,4.896748,"[53efcec39992f4ba59c6aa1a1a58db69, 9029d9c4aff...",Sắc Đẹp,Trang điểm,Trang điểm môi,Son môi,285380605.0,1.900062e+10,ysl


In [ ]:
final.to_excel(r'D:\KiotvietPlus\KiotPro\PROJECT\Shopee-Crawler\data\brand_id_list.xlsx')